In [1]:
import pandas as pd

In [6]:
#df = pd.read_csv(r'parsed_routes_data\all_routes_data.csv') #real dataset
df = pd.read_csv(r'parsed_routes_data\first_bulk_of_routes_data.csv') # smaller sample dataset for testing
df.drop('Unnamed: 1', axis=1, inplace=True)
df.rename(columns={'Unnamed: 0': 'route_id'}, inplace=True)
df.head()

,route_id,lat_lgt,latitude,longitude
0,100324.gpx,"('50.71662', '12.79072')",50.71662,12.79072
1,100324.gpx,"('50.71665', '12.79075')",50.71665,12.79075
2,100324.gpx,"('50.71654', '12.79175')",50.71654,12.79175
3,100324.gpx,"('50.71649', '12.7922')",50.71649,12.79220
4,100324.gpx,"('50.71629', '12.79315')",50.71629,12.79315
